In [1]:
import fiftyone as fo
import os
import pandas as pd
import numpy as np
from glob import glob
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors 
import csv
import json
import sys
import re
from flask import Flask, render_template, request, redirect, url_for
from routes import init_routes
sys.path.append("task-former/code")
from clip.model import convert_weights, CLIP
from clip.clip import _transform, load, tokenize

In [ ]:
# run in about 25 seconds
if fo.dataset_exists("AIC_2024"):
    fo.delete_dataset("AIC_2024")
    
dataset = fo.Dataset.from_images_dir(
    name="AIC_2024", 
    images_dir=os.path.join("..", "data"), 
    recursive=True
)

In [3]:
# run in about 1 minutes 15 seconds
unique_videos = set()
for sample in dataset:
    tmp, sample['video'], sample['keyframe_id'] = sample['filepath'][:-4].rsplit(os.sep, 2)
    sample['batch'] = tmp.rsplit(os.sep, 4)[-3]
    unique_videos.add(sample['video'])
    sample.save()

In [4]:
image_samples = []
image_clip14_embeddings = []
image_task_former_embedding = []
submission_samples = []

In [ ]:
# run in about 4 minutes
video_keyframe_dict = {}
all_keyframe_paths = glob(os.path.join(os.getcwd(), '..', 'data', 'batch*', 'keyframes',
                            '*', '*', '*.jpg'))

video_frameid_dict = {}
for b in [1, 2, 3]:
    for video in unique_videos:
        filepath = os.path.join('..', 'data', f'batch{b}', 'map-keyframes', f'{video}.csv')
        if os.path.exists(filepath):
            a = pd.read_csv(filepath)
            video_frameid_dict[video] = a['frame_idx']

for kf in all_keyframe_paths:
    _, vid, kf = kf[:-4].rsplit(os.sep, 2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

for k, v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

embedding_clip14_dict = {}
embedding_task_former_dict = {}
for j in [1, 2, 3]:
    for video in unique_videos:
        clip14_path = os.path.join('..', 'data', f'batch{j}', 
                            'clip-features-14', f'{video}.npy')
        if os.path.exists(clip14_path):
            a = np.load(clip14_path)
            embedding_clip14_dict[video] = {}
            for i, k in enumerate(video_keyframe_dict[video]):
                embedding_clip14_dict[video][k] = a[i]

        task_former_path = os.path.join('..', 'data', f'batch{j}', 
                    'task-former', f'{video}.npy')
        if os.path.exists(task_former_path):
            b = np.load(task_former_path)
            embedding_task_former_dict[video] = {}
            for i, k in enumerate(video_keyframe_dict[video]):
                embedding_task_former_dict[video][k] = b[i]

for sample in dataset:
    print(sample['video'] + ' - ' + sample['keyframe_id'], end='')
    
    sample['frame_id'] = video_frameid_dict[sample['video']].iloc[int(sample['keyframe_id']) - 1]
    sample['clip-14'] = embedding_clip14_dict[sample['video']][sample['keyframe_id']]
    sample['task-former'] = embedding_task_former_dict[sample['video']][sample['keyframe_id']]
    image_samples.append(sample)
    image_clip14_embeddings.append(sample['clip-14']) 
    image_task_former_embedding.append(sample['task-former'])

    print(" ---  Done")
    sample.save()

In [6]:
image_clip14_embeddings = np.array(image_clip14_embeddings)
image_task_former_embeddings = np.array(image_task_former_embedding)

In [ ]:
# run in 20 seconds
if torch.cuda.is_available():
    device = "cuda"  # Use GPU with CUDA
elif torch.backends.mps.is_available():
    device = "mps"  # Use Metal Performance Shaders for Apple Silicon
else:
    device = "cpu"  # Default to CPU

print(f"Using: {device}")

model_clip14 = CLIPModel.from_pretrained("openai/clip-vit-large-patch14-336").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14-336")

In [61]:
def submission_clip14(text_query, k, csv_file, bias, discard_videos):
    inputs = processor(text=[text_query], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_features = model_clip14.get_text_features(**inputs).cpu().numpy().flatten()
    similarities = cosine_similarity([text_features], image_clip14_embeddings)[0]
    top_k_indices = similarities.argsort()[-k:][::-1]

    if fo.dataset_exists("submission_clip14"):
        fo.delete_dataset("submission_clip14")
    dataset_submission = fo.Dataset(
        name="submission_clip14"
    )

    count = 1
    visited = [False] * k
    for index in range(0, k):
        if (not visited[index]):
            x = []
            x.append(image_samples[top_k_indices[index]])
            visited[index] = True
            for j in range(index + 1, k):
                if (image_samples[top_k_indices[index]]['video'] == image_samples[top_k_indices[j]]['video']
                    and abs(image_samples[top_k_indices[index]]['frame_id'] - image_samples[top_k_indices[j]]['frame_id']) < bias):
                    visited[j] = True
                    x.append(image_samples[top_k_indices[j]])
                    continue
            x = sorted(x, key=lambda a:int(a['frame_id']))
            for e in x:
                e['cluster'] = f'cluster {count}'
                dataset_submission.add_sample(e)
            count += 1

    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['video', 'frame_id'])
        # writer.writeheader()
        for sample in dataset_submission:
            if (sample['video'] not in discard_videos):
                writer.writerow({'video': sample['video'], 'frame_id': sample['frame_id']})

    return dataset_submission

In [ ]:
def submission_clip14_combined(text_query1, weight1, text_query2, weight2, k, csv_file, bias):
    total_weight = weight1 + weight2
    if total_weight == 0:
        raise ValueError("The sum of weight1 and weight2 must not be zero.")
    normalized_weight1 = weight1 / total_weight
    normalized_weight2 = weight2 / total_weight

    inputs1 = processor(text=[text_query1], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_features1 = model_clip14.get_text_features(**inputs1).cpu().numpy().flatten()

    inputs2 = processor(text=[text_query2], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_features2 = model_clip14.get_text_features(**inputs2).cpu().numpy().flatten()

    combined_text_features = (normalized_weight1 * text_features1) + (normalized_weight2 * text_features2)

    similarities = cosine_similarity([combined_text_features], image_clip14_embeddings)[0]
    top_k_indices = similarities.argsort()[-k:][::-1]

    if fo.dataset_exists("submission_clip14_combined"):
        fo.delete_dataset("submission_clip14_combined")
    dataset_submission = fo.Dataset(name="submission_clip14_combined")

    count = 1
    visited = [False] * k
    for index in range(k):
        if not visited[index]:
            cluster_samples = []
            current_index = top_k_indices[index]
            cluster_samples.append(image_samples[current_index])
            visited[index] = True

            for j in range(index + 1, k):
                if not visited[j]:
                    compare_index = top_k_indices[j]
                    if (image_samples[current_index]['video'] == image_samples[compare_index]['video'] and
                        abs(image_samples[current_index]['frame_id'] - image_samples[compare_index]['frame_id']) < bias):
                        cluster_samples.append(image_samples[compare_index])
                        visited[j] = True

            cluster_samples = sorted(cluster_samples, key=lambda x: int(x['frame_id']))
            for sample in cluster_samples:
                sample['cluster'] = f'cluster {count}'
                dataset_submission.add_sample(sample)
            count += 1

    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['video', 'frame_id'])
        # writer.writeheader()  # Uncommented to include headers
        for sample in dataset_submission:
            writer.writerow({'video': sample['video'], 'frame_id': sample['frame_id']})

    return dataset_submission

In [ ]:
def submission_clip14_intersection(text_query1, text_query2, k, csv_file, bias):
    if not isinstance(k, int) or k <= 0:
        raise ValueError("Parameter 'k' must be a positive integer.")

    inputs1 = processor(text=[text_query1], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_features1 = model_clip14.get_text_features(**inputs1).cpu().numpy().flatten()

    similarities1 = cosine_similarity([text_features1], image_clip14_embeddings)[0]
    top_k_indices1 = similarities1.argsort()[-k:][::-1]
    top_k_set1 = set(top_k_indices1)

    inputs2 = processor(text=[text_query2], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_features2 = model_clip14.get_text_features(**inputs2).cpu().numpy().flatten()

    similarities2 = cosine_similarity([text_features2], image_clip14_embeddings)[0]
    top_k_indices2 = similarities2.argsort()[-k:][::-1]
    top_k_set2 = set(top_k_indices2)

    intersection_indices = list(top_k_set1.intersection(top_k_set2))

    if not intersection_indices:
        print("No common images found between the two top-k sets.")
        return fo.Dataset()

    if fo.dataset_exists("submission_clip14_intersection"):
        fo.delete_dataset("submission_clip14_intersection")
    dataset_submission = fo.Dataset(name="submission_clip14_intersection")

    count = 1
    visited = [False] * len(intersection_indices)
    for i in range(len(intersection_indices)):
        if not visited[i]:
            cluster_samples = []
            current_index = intersection_indices[i]
            cluster_samples.append(image_samples[current_index])
            visited[i] = True

            for j in range(i + 1, len(intersection_indices)):
                if not visited[j]:
                    compare_index = intersection_indices[j]
                    if (image_samples[current_index]['video'] == image_samples[compare_index]['video'] and
                        abs(int(image_samples[current_index]['frame_id']) - int(image_samples[compare_index]['frame_id'])) < bias):
                        cluster_samples.append(image_samples[compare_index])
                        visited[j] = True

            cluster_samples = sorted(cluster_samples, key=lambda x: int(x['frame_id']))

            for sample in cluster_samples:
                sample['cluster'] = f'cluster {count}'
                dataset_submission.add_sample(sample)
            count += 1

    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['video', 'frame_id'])
        # writer.writeheader()  # Include headers in the CSV
        for sample in dataset_submission:
            writer.writerow({'video': sample['video'], 'frame_id': sample['frame_id']})

    return dataset_submission

In [ ]:
# run in 11 seconds
model_config_file = os.path.join(os.getcwd(), 'task-former', 'code', 'training', 
                                 'model_configs', 'ViT-B-16.json')
# remember to download file weights .pt from github (or you can contact Thinh Phat)
model_file = os.path.join(os.getcwd(), 'task-former', 'model', 'tsbir_model_final.pt')

with open(model_config_file, 'r') as f:
    model_info = json.load(f)

model_task_former = CLIP(**model_info)
loc = device
checkpoint = torch.load(model_file, map_location=loc)

sd = checkpoint["state_dict"]
if next(iter(sd.items()))[0].startswith('module'):
    sd = {k[len('module.'):]: v for k, v in sd.items()}

model_task_former.load_state_dict(sd, strict=False)
model_task_former.eval()
model_task_former = model_task_former.to(device)

In [10]:
convert_weights(model_task_former)
preprocess_val = _transform(model_task_former.visual.input_resolution, is_train=False)
transformer = preprocess_val
def get_feature(query_sketch, query_text):
    img1 = transformer(query_sketch).unsqueeze(0).to(device)
    txt = tokenize([str(query_text)])[0].unsqueeze(0).to(device)
    
    with torch.no_grad():
        sketch_feature = model_task_former.encode_sketch(img1)
        text_feature = model_task_former.encode_text(txt)
        text_feature = text_feature / text_feature.norm(dim=-1, keepdim=True)
        sketch_feature = sketch_feature / sketch_feature.norm(dim=-1, keepdim=True)

    return model_task_former.feature_fuse(sketch_feature,text_feature)

In [32]:
nbrs = NearestNeighbors(algorithm='brute', metric='cosine').fit(image_task_former_embedding)

def submission_task_former(text_query, sketch_path, k, csv_file, bias):
    sketch = Image.open(sketch_path)
    query_feat = get_feature(sketch, text_query).cpu().numpy()

    nbrs.n_neighbors = k
    distances, indices = nbrs.kneighbors(query_feat)

    if fo.dataset_exists("submission_task_former"):
        fo.delete_dataset("submission_task_former")

    dataset_submission = fo.Dataset(
        name="submission_task_former"
    )

    count = 1
    visited = [False] * k
    for index in range(0, k):
        if (not visited[index]):
            x = []
            x.append(image_samples[indices[0][index]])
            visited[index] = True
            for j in range(index + 1, k):
                if (not visited[j]):
                    if (image_samples[indices[0][index]]['video'] == image_samples[indices[0][j]]['video']
                        and abs(image_samples[indices[0][index]]['frame_id'] - image_samples[indices[0][j]]['frame_id']) < bias):
                        visited[j] = True
                        x.append(image_samples[indices[0][j]])
                        continue
            x = sorted(x, key=lambda a:int(a['frame_id']))
            for e in x:
                e['cluster'] = f'cluster {count}'
                dataset_submission.add_sample(e)
            count += 1

    for index in indices[0]:
        dataset_submission.add_sample(image_samples[index])

    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['video', 'frame_id'])
        # writer.writeheader()
        for sample in dataset_submission:
            writer.writerow({'video': sample['video'], 'frame_id': sample['frame_id']})

    return dataset_submission

In [59]:
nn_model = NearestNeighbors(algorithm='brute', metric='cosine')

def fillResult(csv_file):
    csv_file = os.path.join('..', 'submission', csv_file)
    
    idx_result = []
    final_result = []
    clip14_result = []

    if fo.dataset_exists("submission_fillResult"):
        fo.delete_dataset("submission_fillResult")

    dataset_submission = fo.Dataset(
        name="submission_fillResult"
    )

    df = pd.read_csv(csv_file, header=None, names=['video', 'frame_id'])
    for index, row in df.iterrows():
        for i in range(len(image_samples)):
            if (image_samples[i]['video'] == row['video']
                and image_samples[i]['frame_id'] == row['frame_id']):
                final_result.append(image_samples[i])
                clip14_result.append(image_samples[i]['clip-14'])
                idx_result.append(i)
                continue

    nn_model.n_neighbors = 100
    nn_model.fit(image_clip14_embeddings)
    clip14_result_array = np.array(clip14_result)
    distances, indices = nn_model.kneighbors(clip14_result_array)

    for idx in indices[0]:
        if (idx not in idx_result):
            final_result.append(image_samples[idx])
            dataset_submission.add_sample(image_samples[idx])

    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['video', 'frame_id'])
        # writer.writeheader()
        for i in range(0, 100):
            writer.writerow({'video': final_result[i]['video'], 'frame_id': final_result[i]['frame_id']})

    return dataset_submission

In [13]:
def fillRange(csv_file, video_name, low_frame_id, high_frame_id, mode):
    csv_file = os.path.join('..', 'submission', csv_file)
    with open(csv_file, mode=mode, newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['video', 'frame_id'])
        # writer.writeheader()
        for i in range(low_frame_id, high_frame_id + 1):
            writer.writerow({'video': video_name, 'frame_id': i})

In [101]:
path = []

def help(indices, i, dataset_submission, bias):
    if (i == len(indices)):
        for num in path:
            dataset_submission.add_sample(image_samples[num])
        return
    
    for idx in indices[i]:
        if (not path):
            path.append(idx)
            help(indices, i + 1, dataset_submission, bias)
            path.pop()
        else:
            if (image_samples[idx]['video'] == image_samples[path[-1]]['video']
                 and image_samples[idx]['frame_id'] > image_samples[path[-1]]['frame_id']
                 and image_samples[idx]['frame_id'] - image_samples[path[-1]]['frame_id'] < bias):
                path.append(idx)
                help(indices, i + 1, dataset_submission, bias)
                path.pop()

def search_by_continuous_scene(text_query, k, bias):
    scenes = text_query.split('/ ')
    indices = []
    path.clear()
    for text in scenes:
        inputs = processor(text=[text], return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            text_features1 = model_clip14.get_text_features(**inputs).cpu().numpy().flatten()
        similarities = cosine_similarity([text_features1], image_clip14_embeddings)[0]
        top_k_indices = similarities.argsort()[-k:][::-1]
        indices.append(top_k_indices)

    if fo.dataset_exists("search_by_continuous_scene"):
        fo.delete_dataset("search_by_continuous_scene")

    dataset_submission = fo.Dataset(
        name="search_by_continuous_scene"
    )

    help(indices, 0, dataset_submission, bias)
    
    return dataset_submission

In [42]:
def seePrePost(video_name, frame_id):
    if fo.dataset_exists("pre_and_post"):
        fo.delete_dataset("pre_and_post")

    dataset_submission = fo.Dataset(
        name="pre_and_post"
    )

    closest_index = -1
    diff = 30000

    closest_index = -1
    min_diff = float('inf')

    # Tìm chỉ số với frame_id gần nhất
    for index in range(len(image_samples)):
        if image_samples[index]['video'] == video_name:
            current_frame_id = image_samples[index]['frame_id']
            diff = abs(current_frame_id - frame_id)
            if diff < min_diff:
                min_diff = diff
                closest_index = index

    # Nếu tìm thấy frame_id gần nhất, thêm các mẫu xung quanh nó
    if closest_index != -1:
        low = max(0, closest_index - 10)
        high = min(len(image_samples), closest_index + 10)
        for i in range(low, high):
            dataset_submission.add_sample(image_samples[i])
            
    return dataset_submission

In [ ]:
text_query = "The trailer is about a food festival. However, there is a scene in the trailer where you use virtual reality devices. There is a scene in the trailer where an interviewee is wearing a pink shirt, with a pink backdrop in the background."
output_file = "output.csv"
discard_videos = ['L17_V003']

output_file = os.path.join('..', 'submission', output_file)
dataset_submission = submission_clip14(text_query, 200, output_file, 500, discard_videos)
session = fo.launch_app(dataset_submission, auto=False)
# session.open_tab()

In [ ]:
text_query = 'many people in a stage'
sketch_path = "task-former/my_drawing.png"
output_file = "output.csv"

output_file = os.path.join('..', 'submission', output_file)
dataset_submission = submission_task_former(text_query, sketch_path, k=100, csv_file=output_file, bias=500)
session = fo.launch_app(dataset_submission, auto=False)
# session.open_tab()

In [ ]:
query1 = "The news story tells of the lives of people affected by natural disasters. The news story begins with scenes of many houses destroyed."
query2 = "The news story shows a man in a dark coat and hat visiting a woman in a pink shirt. The news story ends with a rescuer with a blue flashlight on his head trying to save a person buried in the ground."
weight1 = 0.5
weight2 = 0.5

output_file = os.path.join('..', 'submission', output_file)
dataset_submission = submission_clip14_combined(query1, weight1, query2, weight2, 100, 
                                                output_file, 500)
session = fo.launch_app(dataset_submission, auto=False)
# session.open_tab()

In [ ]:
query1 = "many people wearing red scarves"
scene_categories = ['stage', 'performance']
weight1 = 0.5
weight2 = 0.5

query2 = f"A photo of {', '.join(scene_categories)}"
output_file = os.path.join('..', 'submission', output_file)
dataset_submission = submission_clip14_combined(query1, weight1, query2, weight2, 100, 
                                                output_file, 500)
session = fo.launch_app(dataset_submission, auto=False)
session.open_tab()

In [ ]:
query1 = "a stage"
query2 = "there are many students wearing red scarves."

output_file = os.path.join('..', 'submission', output_file)
dataset_submission = submission_clip14_intersection(query1, query2, 1000, output_file, 500)
session = fo.launch_app(dataset_submission, auto=False)
# session.open_tab()

In [ ]:
dataset_submission = fillResult("output.csv")
session = fo.launch_app(dataset_submission, auto=False)
# session.open_tab()

In [18]:
fillRange("query-p1-22-kis.csv", "L12_V013", 7375, 7444, 'w')

In [ ]:
dataset_submission = seePrePost("L10_V016", 6355)
session = fo.launch_app(dataset_submission, auto=False)
# session.open_tab()

In [116]:
text_query = 'The police standing around a person wearing a light brown shirt (with a tattoo on his arm)./ There are 2 people wearing black shirts. The others are wearing yellow reflective vests./ Someone is holding a phone to record./ a police officer working.'
k = 2000
bias = 250
dataset_submission = search_by_continuous_scene(text_query, k, bias)
session = fo.launch_app(dataset_submission, auto=False)
# session.open_tab()

Session launched. Run `session.show()` to open the App in a cell output.


In [20]:
# import re
# def sort_by_middle_number(file_list):
#     def extract_middle_number(filename):
#         match = re.search(r'query-p1-(\d+)-', filename)
#         if match:
#             return int(match.group(1)) 
#         return 0  
    
#     sorted_list = sorted(file_list, key=extract_middle_number)
#     return sorted_list

In [21]:
# SUBMISSION_FOLDER = os.path.join('..', 'submission')

# app = Flask(__name__)
# init_routes(app)

# # Route to handle form submission
# @app.route('/submit_text', methods=['POST'])
# def submit_text():
#     user_text = request.form['inputText']
#     selected_csv_file = request.form.get('file_name')
#     output_file = request.form.get('output_file')
#     k = int(request.form.get('no_images'))
#     bias = int(request.form.get('bias'))
#     notify_interact_text = ''

#     print(bias)

#     csv_files = [f for f in os.listdir(SUBMISSION_FOLDER) if f.endswith('.csv')]
#     csv_files = sort_by_middle_number(csv_files)
#     notify_interact_text = f"Open {selected_csv_file} successfully !"

#     print(user_text)

#     dataset_submission = submission_clip14(user_text, k, os.path.join('..', 'submission', output_file), bias)
#     submission_samples.clear()
#     for sample in dataset_submission:
#         submission_samples.append(sample)
#     session = fo.launch_app(dataset_submission, auto=False)

#     with open(os.path.join('..', 'submission', output_file), newline='', encoding='utf-8') as csvfile:
#         reader = csv.reader(csvfile)
#         csv_content = "\n".join([", ".join(row) for row in reader]) 

#     processed_text = f'Available at <a href="http://localhost:5151/datasets/submission_clip14">http://localhost:5151/datasets/submission_clip14</a>'

#     return render_template('index.html', notify_submit_text=user_text, processed_text=processed_text,
#                            selected_csv_file=selected_csv_file, csv_files=csv_files,
#                            csv_content=csv_content, notify_interact_text=notify_interact_text,
#                            user_text = user_text, output_file=output_file, no_images=k, bias=bias)

# @app.route('/write_selected_imgs_to_file', methods=['POST'])
# def write_selected_imgs_to_file():
#     user_text = request.form['inputText']
#     selected_csv_file = request.form.get('file_name')
#     output_file = request.form.get('output_file')
#     file_name = request.form.get('file_name')
#     bias = int(request.form.get('bias'))
#     k = int(request.form.get('no_images'))
#     csv_file = os.path.join('..', 'submission', file_name)
#     csv_files = [f for f in os.listdir(SUBMISSION_FOLDER) if f.endswith('.csv')]
#     csv_files = sort_by_middle_number(csv_files)

#     if (csv_file.endswith(".csv")):
#         with open(csv_file, mode='w', newline='') as file:
#             writer = csv.DictWriter(file, fieldnames=['video', 'frame_id'])
#             # writer.writeheader()
#             for id in session.selected:
#                 print(id)
#                 for sample in submission_samples:
#                     if (sample['id'] == id):
#                         writer.writerow({'video': sample['video'], 'frame_id': sample['frame_id']})
#                         break
#         with open(csv_file, newline='', encoding='utf-8') as csvfile:
#             reader = csv.reader(csvfile)
#             csv_content = "\n".join([", ".join(row) for row in reader]) 

#     notify_interact_text = f"Write to {file_name} successfully" 
#     return render_template('index.html', notify_submit_text=user_text,
#                            selected_csv_file=selected_csv_file, csv_files=csv_files,
#                            csv_content=csv_content, notify_interact_text=notify_interact_text,
#                            user_text = user_text, output_file=output_file, no_images=k, bias=bias)

# @app.route('/discard_video', methods=['POST'])
# def discard_video():
#     file_name = request.form.get('file_name')
#     discard_video = request.form.get('discard_videos')
#     user_text = request.form.get('inputText')
#     selected_csv_file = request.form.get('file_name')
#     output_file = request.form.get('output_file')
#     bias = int(request.form.get('bias'))
#     k = int(request.form.get('no_images'))
#     csv_files = [f for f in os.listdir(SUBMISSION_FOLDER) if f.endswith('.csv')]
#     csv_files = sort_by_middle_number(csv_files)

#     if (file_name and discard_video):
#         file_path = os.path.join(SUBMISSION_FOLDER, file_name)
#         discard_videos = discard_video.split('\r\n')
#         a = pd.read_csv(file_path, header=None, names=['video', 'frame_id'])
#         with open(file_path, mode='w', newline='') as file:
#             writer = csv.DictWriter(file, fieldnames=['video', 'frame_id'])
#             # writer.writeheader()
#             for i, row in a.iterrows():
#                 if (row['video'] not in discard_videos):
#                     writer.writerow({'video': row['video'], 'frame_id': row['frame_id']})

#         with open(file_path, newline='', encoding='utf-8') as csvfile:
#             reader = csv.reader(csvfile)
#             csv_content = "\n".join([", ".join(row) for row in reader]) 

#         notify_interact_text = f"Discard successfully" 

#     return render_template('index.html', notify_submit_text=user_text,
#                     selected_csv_file=selected_csv_file, csv_files=csv_files,
#                     csv_content=csv_content, notify_interact_text=notify_interact_text,
#                     user_text = user_text, output_file=output_file, no_images=k, bias=bias,
#                     discard_video=discard_video)

# @app.route('/see_pre_and_post', methods=['POST'])
# def see_pre_and_post():
#     file_name = request.form.get('file_name')
#     discard_video = request.form.get('discard_videos')
#     user_text = request.form.get('inputText')
#     selected_csv_file = request.form.get('file_name')
#     output_file = request.form.get('output_file')
#     bias = int(request.form.get('bias'))
#     k = int(request.form.get('no_images'))
#     csv_files = [f for f in os.listdir(SUBMISSION_FOLDER) if f.endswith('.csv')]
#     csv_files = sort_by_middle_number(csv_files)
#     csv_content = request.form.get('csv_edit_text') 

#     video_name = request.form.get('video_name')
#     frame_id = request.form.get('frame_id')
#     dataset_submission = seePrePost(video_name, int(frame_id))
#     submission_samples.clear()
#     for sample in dataset_submission:
#         submission_samples.append(sample)
#     session = fo.launch_app(dataset_submission, auto=False)

#     processed_text_pre_and_post = f'Available at <a href="http://localhost:5151/datasets/pre_and_post">http://localhost:5151/datasets/pre_and_post</a>'

#     return render_template('index.html', notify_submit_text=user_text,
#                     selected_csv_file=selected_csv_file, csv_files=csv_files,
#                     csv_content=csv_content,
#                     user_text = user_text, output_file=output_file, no_images=k, bias=bias,
#                     discard_video=discard_video, processed_text_pre_and_post=processed_text_pre_and_post,
#                     video_name=video_name, frame_id=frame_id)

# app.run(debug=True, use_reloader=False)